## Load Dataset

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
# libgomp issue, must import n2 before torch
from n2 import HnswIndex

In [4]:
import sys

sys.path.insert(0, '../..')

In [5]:
import os
home_dir = os.getenv('HOME')

https://github.com/anhaidgroup/deepmatcher/blob/master/Datasets.md

In [6]:
import glob
import csv
from tqdm.auto import tqdm

from entity_embed.data_utils.utils import Enumerator

row_dict = {}
id_enumerator = Enumerator()
left_id_set = set()
right_id_set = set()
rows_total = 2554 + 22074
clusters_total = 1154

with tqdm(total=rows_total) as pbar:
    with open(f'{home_dir}/Downloads/walmart_amazon_exp_data/exp_data/tableA.csv') as f:
        for row in csv.DictReader(f):
            row['id'] = id_enumerator[f'left-{int(row["id"])}']
            row['source'] = 'left'
            row_dict[row['id']] = row
            left_id_set.add(row['id'])
            pbar.update(1)
    
    with open(f'{home_dir}/Downloads/walmart_amazon_exp_data/exp_data/tableB.csv') as f:
        for row in csv.DictReader(f):
            row['id'] = id_enumerator[f'right-{int(row["id"])}']
            row['source'] = 'right'
            row_dict[row['id']] = row
            right_id_set.add(row['id'])
            pbar.update(1)

In [7]:
train_true_pair_set = set()
valid_true_pair_set = set()
test_true_pair_set = set()

with open(f'{home_dir}/Downloads/walmart_amazon_exp_data/exp_data/train.csv') as f:
    for row in csv.DictReader(f):
        if int(row['label']) == 1:
            id_left = id_enumerator[f'left-{int(row["ltable_id"])}']
            id_right = id_enumerator[f'right-{int(row["rtable_id"])}']
            train_true_pair_set.add((id_left, id_right))

with open(f'{home_dir}/Downloads/walmart_amazon_exp_data/exp_data/valid.csv') as f:
    for row in csv.DictReader(f):
        if int(row['label']) == 1:
            id_left = id_enumerator[f'left-{int(row["ltable_id"])}']
            id_right = id_enumerator[f'right-{int(row["rtable_id"])}']
            valid_true_pair_set.add((id_left, id_right))

with open(f'{home_dir}/Downloads/walmart_amazon_exp_data/exp_data/test.csv') as f:
    for row in csv.DictReader(f):
        if int(row['label']) == 1:
            id_left = id_enumerator[f'left-{int(row["ltable_id"])}']
            id_right = id_enumerator[f'right-{int(row["rtable_id"])}']
            test_true_pair_set.add((id_left, id_right))
        
display(('train_true_pair_set', len(train_true_pair_set)))
display(('valid_true_pair_set', len(valid_true_pair_set)))
display(('test_true_pair_set', len(test_true_pair_set)))

('train_true_pair_set', 576)

('valid_true_pair_set', 193)

('test_true_pair_set', 193)

## Preprocess

In [8]:
attr_list = ['title', 'category', 'brand', 'modelno', 'price']

In [9]:
import unidecode
import itertools
from entity_embed import default_tokenizer

def clean_str(s):
    s = unidecode.unidecode(s).lower().strip()
    s_tokens = itertools.islice((s_part[:30] for s_part in default_tokenizer(s)), 0, 30)
    return ' '.join(s_tokens)[:300]

for row in tqdm(row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

## Init Data Module

In [10]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [11]:
alphabet = list('0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [12]:
attr_info_dict = {
    'title': {
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
    },
    'category': {
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
    },
    'brand': {
        'field_type': "STRING",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'modelno': {
        'field_type': "STRING",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'price': {
        'field_type': "STRING",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
}

In [13]:
from entity_embed import build_row_numericalizer

row_numericalizer = build_row_numericalizer(attr_info_dict, row_dict=row_dict)
row_numericalizer.attr_info_dict

19:29:42 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
19:29:46 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
19:29:49 INFO:For attr='brand', computing actual alphabet and max_str_len
19:29:49 INFO:For attr='brand', using actual_max_str_len=48
19:29:49 INFO:For attr='modelno', computing actual alphabet and max_str_len
19:29:49 INFO:For attr='modelno', using actual_max_str_len=48
19:29:49 INFO:For attr='price', computing actual alphabet and max_str_len
19:29:49 INFO:For attr='price', using actual_max_str_len=10


{'title': NumericalizeInfo(field_type=<FieldType.SEMANTIC_MULTITOKEN: 'semantic_multitoken'>, tokenizer=<function default_tokenizer at 0x7fbb43915dc0>, alphabet=None, max_str_len=None, vocab=<torchtext.vocab.Vocab object at 0x7fbb40d41910>),
 'category': NumericalizeInfo(field_type=<FieldType.SEMANTIC_MULTITOKEN: 'semantic_multitoken'>, tokenizer=<function default_tokenizer at 0x7fbb43915dc0>, alphabet=None, max_str_len=None, vocab=<torchtext.vocab.Vocab object at 0x7fbb40d41d90>),
 'brand': NumericalizeInfo(field_type=<FieldType.STRING: 'string'>, tokenizer=<function default_tokenizer at 0x7fbb43915dc0>, alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=48, vocab=None),
 

In [14]:
from entity_embed import PairwiseDataModule

datamodule = PairwiseDataModule(
    row_dict=row_dict,
    row_numericalizer=row_numericalizer,
    batch_size=10,
    row_batch_size=16,
    train_true_pair_set=train_true_pair_set,
    valid_true_pair_set=valid_true_pair_set,
    test_true_pair_set=test_true_pair_set,
    random_seed=random_seed
)

## Training

In [15]:
from entity_embed import LinkageEmbed

ann_k = 100
model = LinkageEmbed(
    datamodule,
    ann_k=ann_k,
    use_mask=True,
    embedding_size=300
)

In [16]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

max_epochs = 100
early_stop_callback = EarlyStopping(
   monitor='valid_f1_at_0.7',
   min_delta=0.00,
   patience=10,
   verbose=True,
   mode='max'
)
tb_log_dir = '../tb_logs'
tb_name = 'walmart-amazon'
trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    check_val_every_n_epoch=1,
    callbacks=[early_stop_callback],
    logger=TensorBoardLogger(tb_log_dir, name=tb_name)
)

19:29:49 INFO:GPU available: True, used: True
19:29:49 INFO:TPU available: None, using: 0 TPU cores
19:29:49 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [17]:
trainer.fit(model, datamodule)

19:29:51 INFO:
  | Name        | Type       | Params
-------------------------------------------
0 | blocker_net | BlockerNet | 18.5 M
1 | losser      | SupConLoss | 0     
-------------------------------------------
9.6 M     Trainable params
8.9 M     Non-trainable params
18.5 M    Total params


19:29:52 WARNING:true_pair_set has 38 less elements than transitive_true_pair_set. This means there are transitive true pairs not included in true_pair_set.


1

In [18]:
model.blocker_net.get_signature_weights()

{'title': 0.31173351407051086,
 'category': 0.03653138875961304,
 'brand': 0.24241803586483002,
 'modelno': 0.28487545251846313,
 'price': 0.12444164603948593}

## Testing

In [19]:
trainer.test(ckpt_path='best')


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_f1_at_0.3': 0.5203252032520325,
 'test_f1_at_0.5': 0.7612903225806452,
 'test_f1_at_0.7': 0.8273972602739726,
 'test_f1_at_0.9': 0.6115107913669064,
 'test_pair_entity_ratio_at_0.3': 1.4192708333333333,
 'test_pair_entity_ratio_at_0.5': 0.7083333333333334,
 'test_pair_entity_ratio_at_0.7': 0.4479166666666667,
 'test_pair_entity_ratio_at_0.9': 0.22135416666666666,
 'test_precision_at_0.3': 0.3522935779816514,
 'test_precision_at_0.5': 0.6507352941176471,
 'test_precision_at_0.7': 0.877906976744186,
 'test_precision_at_0.9': 1.0,
 'test_recall_at_0.3': 0.9948186528497409,
 'test_recall_at_0.5': 0.917098445595855,
 'test_recall_at_0.7': 0.7823834196891192,
 'test_recall_at_0.9': 0.44041450777202074}
--------------------------------------------------------------------------------


[{'test_precision_at_0.3': 0.3522935779816514,
  'test_recall_at_0.3': 0.9948186528497409,
  'test_f1_at_0.3': 0.5203252032520325,
  'test_pair_entity_ratio_at_0.3': 1.4192708333333333,
  'test_precision_at_0.5': 0.6507352941176471,
  'test_recall_at_0.5': 0.917098445595855,
  'test_f1_at_0.5': 0.7612903225806452,
  'test_pair_entity_ratio_at_0.5': 0.7083333333333334,
  'test_precision_at_0.7': 0.877906976744186,
  'test_recall_at_0.7': 0.7823834196891192,
  'test_f1_at_0.7': 0.8273972602739726,
  'test_pair_entity_ratio_at_0.7': 0.4479166666666667,
  'test_precision_at_0.9': 1.0,
  'test_recall_at_0.9': 0.44041450777202074,
  'test_f1_at_0.9': 0.6115107913669064,
  'test_pair_entity_ratio_at_0.9': 0.22135416666666666}]

## Testing manually 

In [20]:
from entity_embed.data_utils.utils import id_pairs_to_cluster_mapping_and_dict, cluster_dict_to_filtered_row_dict

__, test_cluster_dict = id_pairs_to_cluster_mapping_and_dict(test_true_pair_set)
test_row_dict = cluster_dict_to_filtered_row_dict(row_dict, test_cluster_dict)
test_left_vector_dict, test_right_vector_dict = model.predict(
    row_dict=test_row_dict,
    left_id_set=left_id_set,
    right_id_set=right_id_set,
    batch_size=16
)

In [21]:
embedding_size = model.blocker_net.embedding_size

In [22]:
assert (len(test_left_vector_dict) + len(test_right_vector_dict)) == len(test_row_dict)

In [23]:
%%time

from entity_embed import ANNLinkageIndex

ann_index = ANNLinkageIndex(embedding_size=embedding_size)
ann_index.insert_vector_dict(left_vector_dict=test_left_vector_dict, right_vector_dict=test_right_vector_dict)
ann_index.build()

CPU times: user 213 ms, sys: 0 ns, total: 213 ms
Wall time: 31.1 ms


In [24]:
%%time

sim_threshold = 0.7
found_pair_set = ann_index.search_pairs(
    k=ann_k,
    sim_threshold=sim_threshold,
    left_vector_dict=test_left_vector_dict,
    right_vector_dict=test_right_vector_dict,
)

CPU times: user 219 ms, sys: 0 ns, total: 219 ms
Wall time: 24.1 ms


In [25]:
from entity_embed.evaluation import pair_entity_ratio

pair_entity_ratio(len(found_pair_set), len(test_row_dict))

0.4479166666666667

In [26]:
from entity_embed.evaluation import precision_and_recall, f1_score

precision, recall = precision_and_recall(found_pair_set, test_true_pair_set)
precision, recall

(0.877906976744186, 0.7823834196891192)

In [27]:
f1_score(precision, recall)

0.8273972602739726

In [28]:
false_positives = list(found_pair_set - test_true_pair_set)
len(false_positives)

21

In [29]:
false_negatives = list(test_true_pair_set - found_pair_set)
len(false_negatives)

42

In [30]:
cos_similarity = lambda a, b: np.dot(a, b)

In [31]:
for (id_left, id_right) in false_negatives[:10]:
    display(
        (
            cos_similarity(test_left_vector_dict[id_left], test_right_vector_dict[id_right]),
            row_dict[id_left], row_dict[id_right]
        )
    )

(0.41786113,
 {'id': 932,
  'title': 'da - lite dual vision tensioned cosmopolitan electrol - av format 10 x 10 diagonal',
  'category': 'electronics - general',
  'brand': 'da - lite',
  'modelno': '84992',
  'price': '2735 . 99',
  'source': 'left'},
 {'id': 5367,
  'title': 'da - lite tensioned cosmopolitan electrol - projection screen motorized - 1 1 - dual vision',
  'category': 'projection screens',
  'brand': 'da - lite',
  'modelno': 'cosmopolitan electrol',
  'price': '',
  'source': 'right'})

(0.46397498,
 {'id': 428,
  'title': 'hp 27 black inkjet cartridge c8727an',
  'category': 'printers',
  'brand': 'hp',
  'modelno': 'c8727an',
  'price': '19 . 97',
  'source': 'left'},
 {'id': 9544,
  'title': 'hp 27 black ink cartridge in retail packaging c8727an 140',
  'category': 'inkjet printer ink',
  'brand': 'hp',
  'modelno': 'hewc8727an',
  'price': '18 . 61',
  'source': 'right'})

(0.6783809,
 {'id': 145,
  'title': 'case logic medium slr camera bag',
  'category': 'photography - general',
  'brand': 'case logic',
  'modelno': '133827',
  'price': '47 . 99',
  'source': 'left'},
 {'id': 18445,
  'title': 'case logic slrc - 202 medium slr camera bag black',
  'category': 'cases bags',
  'brand': 'case logic',
  'modelno': 'slrc - 202black',
  'price': '37 . 95',
  'source': 'right'})

(0.6174574,
 {'id': 1883,
  'title': 'prince of persia pc',
  'category': 'software',
  'brand': 'encore',
  'modelno': '21191',
  'price': '10 . 88',
  'source': 'left'},
 {'id': 21677,
  'title': 'prince of persia jc cs',
  'category': 'audio video accessories',
  'brand': 'encore',
  'modelno': '21191',
  'price': '12 . 54',
  'source': 'right'})

(0.4413028,
 {'id': 1613,
  'title': 'pinnacle studio hd v15 ultimate collection pc',
  'category': 'software',
  'brand': 'avid technology',
  'modelno': '',
  'price': '119 . 0',
  'source': 'left'},
 {'id': 19772,
  'title': 'pinnacle studio ultimate collection v . 15',
  'category': 'computers accessories',
  'brand': 'avid',
  'modelno': '82103002401',
  'price': '99 . 99',
  'source': 'right'})

(0.3803586,
 {'id': 2315,
  'title': 'seiko slp - mrl multipurpose label',
  'category': 'stationery & office machinery',
  'brand': 'seiko',
  'modelno': 'slp - mrl',
  'price': '14 . 88',
  'source': 'left'},
 {'id': 12102,
  'title': 'smart label printer multipurpose labels',
  'category': 'audio video accessories',
  'brand': 'seiko',
  'modelno': '',
  'price': '13 . 82',
  'source': 'right'})

(0.54003924,
 {'id': 1810,
  'title': 'microsoft comfort optical mouse 3000 silver blue',
  'category': 'mice',
  'brand': 'microsoft',
  'modelno': 'd1t00011',
  'price': '19 . 95',
  'source': 'left'},
 {'id': 3131,
  'title': 'comfort opt mse3000 silver blu',
  'category': 'computers accessories',
  'brand': 'microsoft',
  'modelno': 'd1t - 00011',
  'price': '11 . 99',
  'source': 'right'})

(0.44987318,
 {'id': 1396,
  'title': 'dj - tech t545a 15 600w 2 - way active loudspeaker',
  'category': 'stereos / audio',
  'brand': 'dj tech',
  'modelno': 't545a',
  'price': '358 . 0',
  'source': 'left'},
 {'id': 4992,
  'title': 'djtech super powered 15 600 w dj spkrs',
  'category': 'home audio theater',
  'brand': 'dj - tech',
  'modelno': 't545a',
  'price': '396 . 74',
  'source': 'right'})

(0.6803147,
 {'id': 2188,
  'title': 'buffalo technology 2tb usb 3 . 0 drivestation duo',
  'category': 'hard drives',
  'brand': 'buffalo technology',
  'modelno': 'hdwl2tu3r1',
  'price': '239 . 82',
  'source': 'left'},
 {'id': 5225,
  'title': 'buffalo technology drivestation duo 2 tb 2 x 1 tb usb 3 . 0 external raid hard drive array hd - wl2tu3r1 black',
  'category': 'computers accessories',
  'brand': 'buffalo technology',
  'modelno': 'hd - wl2tu3r1',
  'price': '249 . 68',
  'source': 'right'})

(0.3146401,
 {'id': 992,
  'title': 'hp 61 black tri - color combo inkjet cartridge',
  'category': 'printers',
  'brand': 'hp',
  'modelno': 'hp 61',
  'price': '30 . 5',
  'source': 'left'},
 {'id': 22146,
  'title': 'hp 61 ink cartridge combo pack',
  'category': 'inkjet printer ink',
  'brand': 'hp',
  'modelno': 'cr259fn # 140',
  'price': '28 . 2',
  'source': 'right'})